In [ ]:

import pandas as pd
from pandas import Series, DataFrame
import numpy as np



## 文本格式数据的读写


In [ ]:

!cat example/ex1.csv


In [ ]:

df = pd.read_csv('example/ex1.csv')
df


In [ ]:

df = pd.read_table('example/ex1.csv', sep=',')
df


In [ ]:

!cat example/ex2.csv


In [ ]:

pd.read_csv('example/ex2.csv', header=None)


In [ ]:

pd.read_csv('example/ex2.csv',
            names=['a', 'b', 'c', 'd', 'message'])


In [ ]:

names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('example/ex2.csv', 
            names=names,
            index_col='message')


In [ ]:

!cat example/csv_mindex.csv


In [ ]:

parsed = pd.read_csv('example/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed


In [ ]:

list(open('example/ex3.txt'))


In [ ]:

result = pd.read_table('example/ex3.txt', sep='\s+')
result


In [ ]:

!cat example/ex4.csv


In [ ]:

pd.read_csv('example/ex4.csv',
            skiprows=[0, 2, 3])


In [ ]:

!cat example/ex5.csv


In [ ]:

result = pd.read_csv('example/ex5.csv')
result


In [ ]:

pd.isnull(result)


In [ ]:

result = pd.read_csv('example/ex5.csv',
                     na_values=['NULL'])
result


In [ ]:

sentinels = {'message': ['foo', 'NA'],
             'something': ['two']}
pd.read_csv('example/ex5.csv', na_values=sentinels)



### 分块读入文本文件


In [ ]:

pd.options.display.max_rows = 10
result = pd.read_csv('example/ex6.csv')
result


In [ ]:

pd.read_csv('example/ex6.csv', nrows=5)


In [ ]:

chunker = pd.read_csv('example/ex6.csv', chunksize=1000)
chunker


In [ ]:

chunker = pd.read_csv('example/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

tot[:10]



### 将数据写入文本格式


In [ ]:

data = pd.read_csv('example/ex5.csv')
data


In [ ]:

data.to_csv('example/out.csv')
!cat example/out.csv


In [ ]:

import sys
data.to_csv(sys.stdout, sep='|')


In [ ]:

data.to_csv(sys.stdout, na_rep='NULL')


In [ ]:

data.to_csv(sys.stdout, index=False, header=False)


In [ ]:

data.to_csv(sys.stdout, index=False, 
            columns=['a', 'b', 'c'])


In [ ]:

dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('example/tseries.csv')
!cat example/tseries.csv 



### 使用分割格式


In [ ]:

!cat example/ex7.csv


In [ ]:

import csv
f = open('example/ex7.csv')
reader = csv.reader(f)


In [ ]:

for line in reader:
    print(line)


In [ ]:

with open('example/ex7.csv') as f:
    lines = list(csv.reader(f))


In [ ]:

header, values = lines[0], lines[1:]


In [ ]:

data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict


In [ ]:

class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

f = open('example/ex7.csv')
reader = csv.reader(f, dialect=my_dialect)


In [ ]:

reader = csv.reader(f, delimiter='|')


In [ ]:

with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))



### JSON  数据


In [ ]:

obj = """
{
    "name": "Wes",
    "places_lived": ["United States", "Spain", "Germany"],
    "pet": null,
    "siblings": [
        {"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
        {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}
    ]
}
"""


In [ ]:

import json



In [ ]:

result = json.loads(obj)
result


In [ ]:

asjson = json.dumps(result)
asjson


In [ ]:

siblings = pd.DataFrame(result['siblings'], 
                        columns=['name', 'age'])
siblings


In [ ]:

!cat example/example.json


In [ ]:

data = pd.read_json('example/example.json')
data


In [ ]:

print(data.to_json())


In [ ]:

print(data.to_json(orient='records'))



### XML, HTML, 网络抓取


In [ ]:

tables = pd.read_html(
    'example/fdic_failed_bank_list.html'
)
len(tables)


In [ ]:

failures = tables[0]
failures.head()


In [ ]:

close_timestamps = pd.to_datetime(
    failures['Closing Date']
)
close_timestamps.dt.year.value_counts()



#### 使用  lxml.objectfy  解析  XML


In [ ]:

from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()


In [ ]:

data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)


In [ ]:

perf = pd.DataFrame(data)
perf.head()


In [ ]:

from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()


In [ ]:

root


In [ ]:

root.get('href')


In [ ]:

root.text



## 二进制格式


In [ ]:

frame = pd.read_csv('example/ex1.csv')
frame


In [ ]:

frame.to_pickle('example/frame_pickle')



### 使用  HDF5  格式


In [ ]:

frame = pd.DataFrame(
    {'a': np.random.randn(1000)}
)

store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store


In [ ]:

store['obj1']


In [ ]:

store.put('obj2', frame, format='table')
store.select('obj2',
             where=['index >= 10 and index <= 15'])


In [ ]:

store.close()


In [ ]:

frame.to_hdf('mydata.h5', 'obj3', 
             format='table')
pd.read_hdf('mydata.h5', 'obj3',
            where=['index < 5'])



### 读取  Microsoft Excel  文件


In [ ]:

xlsx = pd.ExcelFile('example/ex1.xlsx')


In [ ]:

pd.read_excel(xlsx, 'Sheet1')


In [ ]:

frame = pd.read_excel('example/ex1.xlsx', 'Sheet1')
frame


In [ ]:

writer = pd.ExcelWriter('example/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()


In [ ]:

frame.to_excel('example/ex2.xlsx')



## 与  web api  交互


In [ ]:

import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)

resp


In [ ]:

data = resp.json()
data[0]['title']


In [ ]:

issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues



## 与数据库交互


In [ ]:

import sqlite3

query = """
CREATE TABLE test (
    a VARCHAR(20), b VARCHAR(20),
    c REAL       , d INTEGER
)
"""

con = sqlite3.connect('mydata.sqlite')


In [ ]:

con.execute(query)
con.commit()


In [ ]:

data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES (?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()


In [ ]:

cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows


In [ ]:

cursor.description


In [ ]:

pd.DataFrame(rows, 
             columns=[x[0] for x in cursor.description])


In [ ]:

import sqlalchemy as sqla

db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)



## end


In [ ]:

!rm mydata.csv mydata.h5 mydata.sqlite
